<a href="https://colab.research.google.com/github/codewithanirban/Self-Improving-Meta-Learning-for-Autonomous-AI-Architectures/blob/main/Meta_Learning_NAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 786.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
!pip install shimmy>=2.0

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets, transforms
import torch.nn.functional as F
from stable_baselines3 import PPO
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ---------------------
# Step 1: Define the Base CNN Model
# ---------------------
class DynamicCNN(nn.Module):
    def __init__(self, num_layers=3, num_filters=32):
        super(DynamicCNN, self).__init__()
        self.num_layers = num_layers
        self.num_filters = num_filters
        self.layers = self._build_layers()
        self.fc = nn.Linear(self.num_filters * 8 * 8, 10)  # Assuming CIFAR-10, adjust for other datasets

    def _build_layers(self):
        layers = []
        in_channels = 3  # RGB images
        for _ in range(self.num_layers):
            layers.append(nn.Conv2d(in_channels, self.num_filters, kernel_size=3, stride=1, padding=1))
            layers.append(nn.LeakyReLU())
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            in_channels = self.num_filters
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.layers(x)
        x = torch.flatten(x, start_dim=1)
        if x.shape[1] != self.fc.in_features:
          self.fc = nn.Linear(x.shape[1], 10)
        x = self.fc(x)
        return x

In [ ]:
# ---------------------
# Step 2: Define RL-Based Neural Architecture Search (NAS) Controller
# ---------------------
class NASController:
    def __init__(self, env):
        self.model = PPO("MlpPolicy", env, verbose=1)

    def train(self, timesteps=10000):
        self.model.learn(total_timesteps=timesteps)

    def predict(self, state):
        action, _ = self.model.predict(state)
        return action

In [ ]:
# # ---------------------
# # Step 3: Define the Environment for NAS (State, Action, Reward)
# # ---------------------
# class NASEnv:
#     def __init__(self):
#         self.state = [3, 32]  # Initial layers, filters
#         self.action_space = [(1, 16), (1, 32), (-1, -16), (-1, -32)]  # Increase/Decrease layers or filters

#     def step(self, action):
#         change, value = self.action_space[action]
#         if change == 1:
#             self.state[0] = min(self.state[0] + 1, 5)  # Limit layers to 5
#             self.state[1] = min(self.state[1] + value, 128)
#         else:
#             self.state[0] = max(self.state[0] - 1, 1)  # At least 1 layer
#             self.state[1] = max(self.state[1] + value, 16)
#         reward = self.evaluate_model()  # Call function to evaluate model performance
#         return self.state, reward

#     def evaluate_model(self):
#         return np.random.rand()  # Placeholder, replace with actual model performance evaluation


# ---------------------
# Step 3: Define the Environment for NAS (State, Action, Reward) New Code
# ---------------------
import gym
from gym import spaces

class NASEnv(gym.Env): # Inherit from gym.Env
    def __init__(self):
        super(NASEnv, self).__init__() # Call superclass constructor
        self.state = [3, 32]  # Initial layers, filters
        # Define action and observation spaces
        self.action_space = spaces.Discrete(4)  # 4 discrete actions
        self.observation_space = spaces.Box(low=np.array([1, 16]), high=np.array([5, 128]), dtype=np.int32)

    def step(self, action):
        change, value = [(1, 16), (1, 32), (-1, -16), (-1, -32)][action] # Get action from action space
        if change == 1:
            self.state[0] = min(self.state[0] + 1, 5)  # Limit layers to 5
            self.state[1] = min(self.state[1] + value, 128)
        else:
            self.state[0] = max(self.state[0] - 1, 1)  # At least 1 layer
            self.state[1] = max(self.state[1] + value, 16)
        reward = self.evaluate_model()  # Call function to evaluate model performance
        # Assume done is always False for simplicity, adjust as needed
        done = False
        # Info can be an empty dictionary for now
        info = {}
        return np.array(self.state), reward, done, info # Return as numpy array

    def reset(self):
        self.state = [3, 32]  # Reset to initial state
        return np.array(self.state) # Return as numpy array

    def evaluate_model(self):
        return np.random.rand()  # Placeholder, replace with actual model performance evaluation

    # Optional: Define a render method for visualization if needed
    def render(self, mode='human'):
        pass

    # Optional: Define a close method for cleanup if needed
    def close(self):
        pass

In [ ]:

# ---------------------
# Step 4: Integrate Meta-Learning (MAML)
# ---------------------
class MAML:
    def __init__(self, model, lr=0.01, inner_steps=1):
        self.model = model
        self.lr = lr
        self.inner_steps = inner_steps

    def adapt(self, loss):
        grads = torch.autograd.grad(loss, self.model.parameters(), create_graph=True)
        for param, grad in zip(self.model.parameters(), grads):
            param.data -= self.lr * grad

    def train_meta(self, train_loader, criterion, optimizer):
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = self.model(images)
            loss = criterion(outputs, labels)
            self.adapt(loss)
            optimizer.step()


In [ ]:
# ---------------------
# Step 5: Training Pipeline with Metrics
# ---------------------
def train_model():
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),  #Add random horizontal flip
        transforms.RandomRotation(10),  #Add random rotation
        transforms.ToTensor(),
        ])
    train_data = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    test_data = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

    model = DynamicCNN()
    maml = MAML(model)
    env = NASEnv()
    nas = NASController(env)
    optimizer = optim.AdamW(model.parameters(), lr=1e-10) #Changed learning rate
    criterion = nn.CrossEntropyLoss()

    for epoch in range(10):
        for images, labels in train_loader:

            outputs = model(images)
            loss = criterion(outputs, labels)
            maml.adapt(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        action = nas.predict(env.state)
        env.step(action)
        print(f"Epoch {epoch}: Updated Architecture - {env.state}")

    # Evaluate on test data
    y_true, y_pred = [], []
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    print(f"Test Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
train_model()

Files already downloaded and verified
Files already downloaded and verified
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Epoch 0: Updated Architecture - [4, 64]
Epoch 1: Updated Architecture - [3, 32]
Epoch 2: Updated Architecture - [4, 48]
Epoch 3: Updated Architecture - [5, 64]
Epoch 4: Updated Architecture - [5, 80]
Epoch 5: Updated Architecture - [5, 112]
Epoch 6: Updated Architecture - [4, 80]
Epoch 7: Updated Architecture - [5, 96]
Epoch 8: Updated Architecture - [4, 80]
Epoch 9: Updated Architecture - [3, 64]
Test Accuracy: 0.4976, Precision: 0.5064, Recall: 0.4976, F1-score: 0.4786
